In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup as Soup
import pandas as pd
import mysql.connector
import sqlalchemy
import time
import requests

In [2]:
city_to_scrap = ['Paris', 'Lyon', 'Toulouse', 'Nantes', 'Bordeaux']
work_to_scrap = ['développeur', 'data scientist', 'data analyst', 'business intelligence']

In [3]:
df = pd.DataFrame(columns=["Titre","Entreprise","Ville","Salaire","Type de contrat", "Descriptif du poste", "Date de publication"])

In [4]:
def starting(job, ville):
    #Choisir localisation(temps d'attente pour le click)
    entrer_job = browser.find_element_by_xpath('//*[@id="text-input-what"]')
    entrer_job.send_keys(job)
    time.sleep(1)
    #Ecrire ville
    entrer_city = browser.find_element_by_xpath('//*[@id="text-input-where"]')
    entrer_city.send_keys(ville)
    time.sleep(1)
    #Recherche 
    button_search = browser.find_element_by_class_name("icl-Button")
    button_search.click()
    time.sleep(2)

In [5]:
def restart_or_continue():
    buttons = browser.find_elements_by_class_name("np")
    if len(buttons) == 1:
        # If
        if 'Précédent' in buttons[0].text:
            return_to_first_page()
            browser.quit()
        next_button = buttons[0]
    # If not first page
    else:
        next_button = buttons[1]
    next_button.click()
def return_to_first_page():
    #block à exécuter si on se trouve sur la dernière page des résultats
    browser.get('https://www.indeed.fr/')
    #Clean les zones de texte
    browser.find_element_by_xpath('//*[@id="text-input-what"]').clear()
    browser.find_element_by_xpath('//*[@id="text-input-where"]').clear()
            
#browser.quit() #Attention ça ferme la fênetre

In [6]:
browser = webdriver.Chrome()
browser.get('https://www.indeed.fr/')
browser.maximize_window()

In [7]:
def process_scrapping():
    starting(work,city)
        while True:
            #éviter le popup
            try:
                popUp = browser.find_element_by_xpath('//*[@id="popover-x"]/a')
                popUp.click()
                time.sleep(0.5)
            except: #block à exécuter si le try ne fonctionne pas
                pass
            for chaque in browser.find_elements_by_class_name('result'):
                chaque.find_element_by_class_name('jobtitle').click()
                time.sleep(0.3)
                try:
                    title = chaque.find_element_by_class_name('jobtitle').text
                except:
                    title = 'None'
                #Scraping le nom de la boite sinon rien 
                try:
                    boite = chaque.find_element_by_class_name('company').text
                except:
                    boite = 'Nane'
                #Scraping le Ville sinon rien 
                try:
                    city = chaque.find_element_by_class_name('location').text
                except:
                    city = 'None'
                #Scraping le salaire sinon rien 
                try:
                    salary = chaque.find_element_by_class_name('salaryText').text
                except:
                    salary = 'None'
                #Scraping le type de contrat sinon chercher une autre position 
                try:
                    if len(browser.find_elements_by_class_name('jobMetadataHeader-itemWithIcon-label')) == 2:
                        if browser.find_elements_by_class_name('jobMetadataHeader-itemWithIcon-label')[1] != salary:
                            contrat = browser.find_element_by_class_name('jobMetadataHeader-itemWithIcon-label')[1].text
                    elif len(browser.find_elements_by_class_name('jobMetadataHeader-itemWithIcon-label')) == 3:
                        if browser.find_elements_by_class_name('jobMetadataHeader-itemWithIcon-label')[1] != salary:
                            contrat = browser.find_element_by_class_name('jobMetadataHeader-itemWithIcon-label')[1].text
                    if salary == contrat:
                        contrat = None
                except: 
                    try:
                        contrat = browser.find_element_by_xpath('//*[@id="vjs-tab-job"]/div[1]/div[2]/span[2]').text
                    except:
                        contrat = None
                #Scraping le description sinon rien 
                try:
                    describe = chaque.find_element_by_class_name('summary').text
                except:
                    describe = 'None'
                #Scraping le date de publication sinon rien 
                try:
                    publish_date = browser.find_element_by_xpath('//*[@id="vjs-footer"]/div[1]/div/span[1]').text
                except:
                    publish_date = 'None'
                df = df.append({"Titre" : title,"Entreprise" : boite,"Ville" : city,"Salaire": salary, "Type de contrat" : contrat, "Descriptif du poste" : describe, "Date de publication" : publish_date}, ignore_index=True)
            restart_or_continue()

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=51324): Max retries exceeded with url: /session/e2722d22ebbfd7dbf292cf0dcd3c5164/element/247e5b79-0942-4286-ae1b-6a8b99ab4fbd/click (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11ce12898>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
#Fonction pour les villes
def loop_for_city():
for city in city_to_scrap:
    i =0
    loop_for_work()
    i += 1

In [ ]:
#Fonction pour les jobs
def loop_for_work():
    for work in work_to_scrap:
        k = 0
        process_scrapping()
        k += 1

In [8]:
df

,Titre,Entreprise,Ville,Salaire,Type de contrat,Descriptif du poste,Date de publication
0,Customer Success Officer,Chronotruck,Paris 1er (75),28 000 € - 33 000 € par an,"Temps plein, CDI",Chronotruck est une plateforme web et mobile d...,il y a 30+ jours
1,"Stage ""Business Dev & Growth Hacking"" [#Startu...",Superpitch,Paris 11e (75),None,Stage,Superpitch est une jeune agence créative dédié...,il y a 9 mois
2,Commercial H/F,H2mat',Île-de-France,35 000 € par an,"Temps plein, CDI",En tant que Chargé(e) de dév. Vous souhaitez f...,il y a 6 jours
3,QA Analyst,Praxedo,Paris 9e (75),None,"Temps plein, CDI","Au sein de l'équipe technique, vous intégrez u...",il y a 30+ jours
4,Webmaster / Webdesigner/ Informatique,Studio des Parfums,Paris 4e (75),None,"Temps plein, Temps partiel, CDD, Stage, Freela...",Studio de création spécialisé en parfum sur me...,il y a 2 jours
5,Fleet Manager - Junior,deliver.ee,Paris (75),700 € par mois,Stage,Tu travailleras avec les différentes équipes (...,il y a 30+ jours
6,PYTHON DEV - F/H,Ingeniance,Paris (75),None,CDI,De l'écriture des normes de dev de l'équipe. N...,il y a 1 jour
7,DEV FULLSTACK PHP SF |PARIS 16E | BELLE OPPORT...,In-Team,Paris (75),43 000 € - 75 000 € par an,43 000 € - 75 000 € par an,Parce que votre passion pour les technologies ...,il y a 11 mois
8,DEV WEB REACT - CDI,Kannelle,Paris (75),35 000 € - 55 000 € par an,35 000 € - 55 000 € par an,‍ ‍ ‍ Tu rejoindras une équipe tech de 4 perso...,Station F
9,Dev Full Stack ��,Libon,Paris (75),4 000 € - 5 500 € par mois,"Temps plein, CDI","Dev full stack ou similaire: Par ailleurs, nou...",il y a 13 jours


In [ ]:
loop_for_city()